<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Inits" data-toc-modified-id="Imports-&amp;-Inits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports &amp; Inits</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Loading</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Misc" data-toc-modified-id="Misc-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Misc</a></span></li></ul></div>

# Surname Classifier Using ElmanRNN

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import pdb
import pandas as pd
import numpy as np
import torch
import re

from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

from ignite.engine import Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [3]:
from surname.dataset import SurnameDataset
from surname.containers import DataContainer, ModelContainer
from surname.model import SurnameClassifier
from surname.trainer import IgniteTrainer
from consts import consts
vars(consts)

{'path': PosixPath('../data/surnames'),
 'workdir': PosixPath('../data/surnames/rnn_workdir'),
 'proc_dataset_csv': PosixPath('../data/surnames/surnames_with_splits.csv'),
 'model_dir': PosixPath('../data/surnames/rnn_workdir/models'),
 'vectorizer_json': PosixPath('../data/surnames/rnn_workdir/elman_vectorizer.json'),
 'metrics_file': PosixPath('../data/surnames/rnn_workdir/elman_metrics.csv'),
 'class_weights_pth': PosixPath('../data/surnames/rnn_workdir/class_weights.pth'),
 'char_embedding_sz': 100,
 'rnn_hidden_sz': 64,
 'bs': 64,
 'lr': 0.001,
 'n_epochs': 97,
 'device': 'cuda:3',
 'checkpointer_prefix': 'elman',
 'checkpointer_name': 'classifier',
 'es_patience': 11,
 'save_every': 2,
 'save_total': 5}

## Data Loading

In [ ]:
df = pd.read_csv(consts.proc_dataset_csv)
print(df.shape)
df.head()

In [ ]:
dc = DataContainer(df, SurnameDataset, consts.vectorizer_json, consts.bs, is_load=True)

In [ ]:
try:
  class_weights = torch.load(consts.class_weights_pth)
except FileNotFoundError:
  nationality_vocab = dc.nationality_vocab
  class_counts = df['nationality'].value_counts().to_dict()
  sorted_counts = sorted(class_counts.items(), key=lambda x: nationality_vocab.lookup_token(x[0]))
  freq = [count for _, count in sorted_counts]
  class_weights = 1.0/torch.tensor(freq, dtype=torch.float32)
  torch.save(class_weights, consts.class_weights_pth)

## Model

In [ ]:
classifier = SurnameClassifier(consts.char_embedding_sz, dc.vocab_size, dc.n_classes, consts.rnn_hidden_sz, \
                       padding_idx=dc.surname_vocab.mask_idx)
class_weights = class_weights.to(consts.device)
loss_fn = nn.CrossEntropyLoss(class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=consts.lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 0.5, patience=1)
mc = ModelContainer(classifier, optimizer, loss_fn, scheduler)
mc.model

In [ ]:
itr = iter(dc.train_dl)

In [ ]:
inp,y = next(itr)
# y_pred = mc.model(x,l)
# loss_fn(y_pred, y)

In [ ]:
f(*inp)

In [ ]:
def f(x_in, x_lens, apply_softmax=False):
#   x_in = args[0][0]
#   x_lens = args[0][1]
#   x_in = args[0]
#   x_lens = args[1]
  
  print(x_in.shape, x_lens.shape)

## Training

In [ ]:
pbar = ProgressBar(persist=True)
metrics = {'accuray': Accuracy(), 'loss': Loss(loss_fn)}

In [ ]:
consts.n_epochs=2
ig = IgniteTrainer(mc, dc, consts, pbar, metrics)

In [ ]:
ig.run()

In [ ]:
%debug

## Misc

In [ ]:
bs=3
hidden_sz=7
seq_sz =5 

In [ ]:
x_lens = torch.randint(1, seq_sz+1, (bs,))
x_lens = x_lens.long().detach().cpu().numpy()-1
y_out = torch.randn(bs, seq_sz, hidden_sz)

In [ ]:
print(x_lens.shape)
x_lens

In [ ]:
print(y_out.shape)
y_out

In [ ]:
out = []

for batch_idx, column_idx in enumerate(x_lens):
  out.append(y_out[batch_idx, column_idx])
#   print(batch_idx, column_idx)

In [ ]:
y = torch.stack(out)

In [ ]:
print(y.shape)
y

In [ ]:
bs=3
hidden_sz=7
seq_sz =5 

In [4]:
from surname.elman import ElmanRNN
e = ElmanRNN(consts.char_embedding_sz, consts.rnn_hidden_sz, batch_first=True)

In [5]:
inp = torch.randn(2,10,100)
e(inp)

tensor([[[-0.5691,  0.9601, -0.1866,  ...,  0.2022, -0.1335, -0.5653],
         [ 0.2962, -0.1956,  0.7913,  ...,  0.1010,  0.0103, -0.7296],
         [ 0.7989,  0.5877,  0.5918,  ...,  0.6926, -0.5163, -0.3968],
         ...,
         [ 0.4169,  0.7637, -0.6288,  ..., -0.5383,  0.8708,  0.4124],
         [-0.6200, -0.5039, -0.1494,  ..., -0.3323,  0.7859, -0.7193],
         [ 0.0745, -0.3591,  0.0318,  ...,  0.0753,  0.5478, -0.8766]],

        [[-0.8320, -0.3385, -0.4728,  ..., -0.6391,  0.3053, -0.8308],
         [ 0.3537, -0.7650,  0.8903,  ..., -0.3771, -0.0213, -0.0107],
         [-0.3165,  0.8988, -0.3397,  ...,  0.4064, -0.9452,  0.8035],
         ...,
         [-0.0522,  0.3082, -0.6157,  ...,  0.2660, -0.0645,  0.2667],
         [ 0.6754,  0.5227,  0.5624,  ..., -0.6347, -0.5460,  0.1425],
         [-0.6997, -0.4942, -0.5160,  ..., -0.8357, -0.2817, -0.9316]]],
       grad_fn=<PermuteBackward>)

In [6]:
inp = inp.to('cuda:3')
e = e.to('cuda:3')

In [ ]:
inp = inp.cpu()
e = e.cpu()

In [7]:
e(inp)

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #4 'mat1'

In [ ]:
x = torch.zeros(5, device='cuda:3')

In [ ]:
x